<a href="https://colab.research.google.com/github/karimNafiz/kaggle_competition_comp432/blob/main/comp432_kaggle_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


In [108]:
from torch.nn.modules import linear

In [107]:
import math

In [60]:
from torch.utils.data import TensorDataset, DataLoader, random_split

In [61]:
import matplotlib.pyplot as plt

In [64]:
# this line is important, I was using the default C reader, which is less fault tolerant, so I was reading less data
df = pd.read_csv('train.csv', engine='python', quotechar='"', escapechar='\\')


(115406, 502)

In [65]:
df.shape

(115406, 502)

In [66]:
drop_column = ['id']


In [95]:

df = df.drop(drop_column, axis = 1)

In [103]:
batch_size = 64
raw_width = 25
raw_height = 20

In [77]:
df.shape

(115406, 501)

In [78]:
data_np = df.values

In [79]:
data_np.shape

(115406, 501)

In [80]:
full_data = torch.tensor(data_np)

In [81]:
full_data.shape

torch.Size([115406, 501])

In [82]:
N = full_data.size(0)


In [83]:
val_ratio = 0.25
val_size = int(val_ratio * N)
train_size = N - val_size

In [84]:
indices = torch.randperm(N)

In [85]:
val_idx = indices[:val_size]
train_idx = indices[val_size:]

In [86]:
train_full = full_data[train_idx]
val_full = full_data[val_idx]

In [87]:
train_full.size()

torch.Size([86555, 501])

In [88]:
val_full.size()

torch.Size([28851, 501])

In [89]:
# just making sure, everything is there
N == val_full.shape[0] + train_full.shape[0]

True

In [91]:
# splitting features and target
train_X = train_full[:,:-1]
train_y = train_full[:,-1]


In [92]:
train_y = train_y.long()

In [93]:
val_X = val_full[:,:-1]
val_y = val_full[:,-1]

In [94]:
val_y = val_y.long()

In [97]:
# need to reshape the data
train_X_2D = train_X.reshape(-1, raw_height, raw_width)

In [98]:
val_X_2D = val_X.reshape(-1,raw_height, raw_width)

In [101]:
# just making sure, everything went well
N == train_X_2D.shape[0] + val_X_2D.shape[0]

True

In [102]:
# making the tensor datasets
training_dataset = TensorDataset(train_X_2D, train_y)
val_dataset = TensorDataset(val_X_2D, val_y)

In [105]:
# making the data Loaders imp for training
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

In [106]:
#!!!! need to normalize the fcking data

In [109]:

class HouseObjectClassifier(nn.Module):
  def __init__(self, input_channels, image_height, image_width, output_class):
    super().__init__()
    self.convolution_layer1 = nn.Conv2d(input_channels,16,kernel_size=3, padding=1)
    self.maxPool1 = nn.MaxPool2d(2,2,ceil_mode=True)
    self.convolution_layer2 = nn.Conv2d(16,32,kernel_size=3,padding=1)
    self.maxPool2 = nn.MaxPool2d(2,2,ceil_mode=True)
    # after this the image should have the shape 32, math.ceil(height/4), math.ceil(width/4)
    self.linear_layer1 = nn.Linear(32*math.ceil(image_height/4)*math.ceil(image_width/4), 512)
    self.linear_layer2 = nn.Linear(512, output_class)

  def forward(self, x):
    x = self.convolution_layer1(x)
    x = torch.relu(x)
    x = self.maxPool1(x)
    x = self.convolution_layer2(x)
    x = torch.relu(x)
    x = self.maxPool2(x)

    # fully connected layer starts
    x = x.view(x.size(0),-1)
    x = self.linear_layer1(x)
    x = torch.relu(x)
    x = self.linear_layer2(x)
    return x



In [110]:
               # should be [4, 10]


torch.Size([4, 10])
